# Marcação a Mercado no Tesouro Direto

This project performs a mark-to-market analysis of Brazilian Treasury bonds. The goal is to identify cycles of appreciation and depreciation using quartile-based statistical analysis, with Python automating the ETL process and Power BI used for data visualization.

## 1. Installing Libraries

In [1]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=f91650529c7c24e1f9977a38b5ea87ac1086ebdb1d8f1731e362df9ea6747e33
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


## 2. Importing Libraries

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import wget

from scipy.stats import poisson
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.manifold import TSNE
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import time
from datetime import datetime
from requests import get
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

## 3. Authentication and Access to Google Drive

In [7]:
url_tesouro = 'https://www.tesourotransparente.gov.br/ckan/dataset/taxas-dos-titulos-ofertados-pelo-tesouro-direto/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1?_gl=1*1wqohxr*_gcl_au*OTI4MjI5ODE5LjE3MTQ0MTg5MDU.*_ga*MTQ3NTU3NjUzMS4xNzE0NDE4OTA1*_ga_95FH8RQ7M0*MTcyMDYzMzk0Mi4xMC4xLjE3MjA2MzM5NzQuMjguMC4w'
format = '.csv'

In [8]:
text_tesouro = BeautifulSoup(requests.get(url_tesouro).text, 'html.parser')
text_tesouro

<!DOCTYPE html>

<!--[if IE 7]> <html lang="pt_BR" class="ie ie7"> <![endif]-->
<!--[if IE 8]> <html lang="pt_BR" class="ie ie8"> <![endif]-->
<!--[if IE 9]> <html lang="pt_BR" class="ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="pt_BR"> <!--<![endif]-->
<head>
<!--[if lte ie 8]><script type="text/javascript" src="/ckan/fanstatic/vendor/:version:2015-07-09T12:08:55.16/html5.min.js"></script><![endif]-->
<link href="/ckan/fanstatic/vendor/:version:2015-07-09T12:08:55.16/select2/select2.css" rel="stylesheet" type="text/css"/>
<link href="/ckan/fanstatic/css/:version:2015-07-09T12:08:55.06/main.min.css" rel="stylesheet" type="text/css"/>
<link href="/ckan/fanstatic/vendor/:version:2015-07-09T12:08:55.16/font-awesome/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<!--[if ie 7]><link rel="stylesheet" type="text/css" href="/ckan/fanstatic/vendor/:version:2015-07-09T12:08:55.16/font-awesome/css/font-awesome-ie7.min.css" /><![endif]-->
<meta charset="utf-8"/>
<meta conten

## 4. Data Collection and Loading

In [9]:
link = text_tesouro.find_all('a', href = True, class_= "btn btn-primary resource-url-analytics resource-type-None"  )
link

[<a class="btn btn-primary resource-url-analytics resource-type-None" href="https://www.tesourotransparente.gov.br/ckan/dataset/df56aa42-484a-4a59-8184-7676580c81e3/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1/download/PrecoTaxaTesouroDireto.csv">
 <i class="icon-download-alt"></i> Download do Arquivo
                   
                 </a>]

In [10]:
links = [l.get("href") for l in link]
links[0]

'https://www.tesourotransparente.gov.br/ckan/dataset/df56aa42-484a-4a59-8184-7676580c81e3/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1/download/PrecoTaxaTesouroDireto.csv'

In [11]:
url = "https://www.tesourodireto.com.br/json/br/com/b3/tesourodireto/service/api/treasurybondsinfo.json"

# Fazendo a requisição GET com verificação SSL desativada
response = requests.get(url, verify=False)
response

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tesourodireto.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [12]:
url = "https://www.tesourodireto.com.br/titulos/precos-e-taxas.htm"
response = requests.get(url)



In [13]:
# from bs4 import BeautifulSoup
# import requests
# import pandas as pd

# # URL do HTML
# url = "https://www.tesourodireto.com.br/titulos/precos-e-taxas.htm"
# response = requests.get(url)

# # Parseando o HTML com BeautifulSoup
# soup = BeautifulSoup(response.content, 'html.parser')

# # Encontrando a tabela
# table = soup.find('table', class_='td-invest-table')

# # Extraindo os cabeçalhos
# headers = [th.text.strip() for th in table.find_all('th')]

# # Extraindo os dados das linhas
# rows = []
# for tr in table.find_all('tr')[1:]:
#     cells = tr.find_all('td')
#     row = [cell.text.strip() for cell in cells if cell.text.strip()]
#     if row:
#         rows.append(row)

# # Convertendo para DataFrame
# df = pd.DataFrame(rows, columns=headers)

# # Exibindo a tabela
# print(df)


## 5. Data Cleaning and Transformation

In [14]:
# tabela = text_tesouro_tabela.find_all('r')
# tabela

In [15]:
# tabelas = [t.get("span") for t in tabela]
# tabelas

In [16]:
response = wget.download(links[0])

In [17]:
# text = BeautifulSoup(requests.get(url).text, 'html.parser')
# link = text.find_all('a', href = True, string="Premier League" )
# links = [l.get("href") for l in link]
# response = wget.download(domain + '/' +  links[0])

In [18]:
df = pd.read_csv('PrecoTaxaTesouroDireto.csv', sep = ';')
df

,Tipo Titulo,Data Vencimento,Data Base,Taxa Compra Manha,Taxa Venda Manha,PU Compra Manha,PU Venda Manha,PU Base Manha
0,Tesouro IPCA+ com Juros Semestrais,15/05/2015,01/12/2006,"8,34","8,40","1398,76","1393,68","1392,76"
1,Tesouro IPCA+ com Juros Semestrais,15/05/2011,01/12/2006,"8,64","8,68","1468,22","1466,10","1465,12"
2,Tesouro IPCA+ com Juros Semestrais,15/08/2010,01/12/2006,"8,63","8,67","1513,00","1511,17","1510,16"
3,Tesouro IPCA+,15/05/2015,01/12/2006,"8,30","8,36","822,85","819,03","818,49"
4,Tesouro IPCA+,15/08/2024,01/12/2006,"7,73","7,81","433,17","427,54","427,27"
...,...,...,...,...,...,...,...,...
155008,Tesouro IPCA+,15/05/2015,06/10/2006,"8,83","8,89","775,41","771,75","771,33"
155009,Tesouro IPCA+,15/08/2024,06/10/2006,"7,90","7,98","414,18","408,76","408,55"
155010,Tesouro IGPM+ com Juros Semestrais,01/07/2017,06/10/2006,"8,62","8,70","1570,26","1561,49","1560,63"
155011,Tesouro IGPM+ com Juros Semestrais,01/12/2006,06/10/2006,"11,41","11,43","1984,06","1984,01","1982,72"


In [19]:
df['ano_vencimento'] = "0"
df['ano_analisado'] = "0"

for i in range(len(df)):
  df['ano_vencimento'][i] = df['Data Vencimento'][i][6:]
  df['ano_analisado'][i] = df['Data Base'][i][6:]

Streaming output truncated to the last 5000 lines.

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['ano_analisado'][i] = df['Data Base'][i][6:]
<ipython-input-17-7d62ba4858f7>:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perfor

In [20]:
df['ano_vencimento'].value_counts()
df['ano_analisado'].value_counts()

,count
ano_analisado,
2024,13812
2023,10689
2008,9529
2007,9048
2009,8429
2006,8288
2010,8003
2011,7209
2022,7145


In [21]:
df["ano_analisado"] = pd.to_numeric(df["ano_analisado"])
df["ano_analisado"]

,ano_analisado
0,2006
1,2006
2,2006
3,2006
4,2006
...,...
155240,2005
155241,2005
155242,2005
155243,2005


In [22]:
ano_corte = df["ano_analisado"].max() - 10

In [23]:
df = df[df['ano_analisado'] >= ano_corte]
df = df.reset_index(drop=True)
df

,Tipo Titulo,Data Vencimento,Data Base,Taxa Compra Manha,Taxa Venda Manha,PU Compra Manha,PU Venda Manha,PU Base Manha,ano_vencimento,ano_analisado
0,Tesouro Prefixado,01/01/2031,10/01/2025,"15,08","15,20","434,86","431,94","431,94",2031,2025
1,Tesouro Prefixado,01/01/2027,10/01/2025,"15,31","15,43","756,34","754,37","754,37",2027,2025
2,Tesouro Prefixado,01/01/2026,10/01/2025,"14,92","15,04","873,53","872,16","872,16",2026,2025
3,Tesouro Prefixado,01/01/2029,10/01/2025,"15,19","15,31","572,79","570,12","570,12",2029,2025
4,Tesouro IPCA+ com Juros Semestrais,15/05/2055,10/01/2025,"7,32","7,44","3768,88","3711,94","3711,94",2055,2025
...,...,...,...,...,...,...,...,...,...,...
78748,Tesouro IGPM+ com Juros Semestrais,01/04/2021,02/01/2015,"6,12","6,18","3073,37","3064,35","3062,05",2021,2015
78749,Tesouro Prefixado com Juros Semestrais,01/01/2023,02/01/2015,"12,43","12,49","887,58","884,96","884,55",2023,2015
78750,Tesouro Prefixado com Juros Semestrais,01/01/2017,02/01/2015,"12,99","13,05","953,13","952,20","951,74",2017,2015
78751,Tesouro Prefixado com Juros Semestrais,01/01/2025,02/01/2015,"12,34","12,40","876,52","873,57","873,17",2025,2015


In [24]:
df["Nome_titulo"] = 'a'

for i in range(len(df)):
  df['Nome_titulo'][i] = df['Tipo Titulo'][i] + ' ' + df['Data Vencimento'][i][6:]

Streaming output truncated to the last 5000 lines.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Nome_titulo'][i] = df['Tipo Titulo'][i] + ' ' + df['Data Vencimento'][i][6:]
<ipython-input-22-7c294844b537>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Nome_titulo'][i] = df['Tipo Titulo'][i] + ' ' + df['Data Vencimento'][i][6:]
<ipython-input-22-7c294844b537>:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate o

In [25]:
df

,Tipo Titulo,Data Vencimento,Data Base,Taxa Compra Manha,Taxa Venda Manha,PU Compra Manha,PU Venda Manha,PU Base Manha,ano_vencimento,ano_analisado,Nome_titulo
0,Tesouro Prefixado,01/01/2031,10/01/2025,"15,08","15,20","434,86","431,94","431,94",2031,2025,Tesouro Prefixado 2031
1,Tesouro Prefixado,01/01/2027,10/01/2025,"15,31","15,43","756,34","754,37","754,37",2027,2025,Tesouro Prefixado 2027
2,Tesouro Prefixado,01/01/2026,10/01/2025,"14,92","15,04","873,53","872,16","872,16",2026,2025,Tesouro Prefixado 2026
3,Tesouro Prefixado,01/01/2029,10/01/2025,"15,19","15,31","572,79","570,12","570,12",2029,2025,Tesouro Prefixado 2029
4,Tesouro IPCA+ com Juros Semestrais,15/05/2055,10/01/2025,"7,32","7,44","3768,88","3711,94","3711,94",2055,2025,Tesouro IPCA+ com Juros Semestrais 2055
...,...,...,...,...,...,...,...,...,...,...,...
78748,Tesouro IGPM+ com Juros Semestrais,01/04/2021,02/01/2015,"6,12","6,18","3073,37","3064,35","3062,05",2021,2015,Tesouro IGPM+ com Juros Semestrais 2021
78749,Tesouro Prefixado com Juros Semestrais,01/01/2023,02/01/2015,"12,43","12,49","887,58","884,96","884,55",2023,2015,Tesouro Prefixado com Juros Semestrais 2023
78750,Tesouro Prefixado com Juros Semestrais,01/01/2017,02/01/2015,"12,99","13,05","953,13","952,20","951,74",2017,2015,Tesouro Prefixado com Juros Semestrais 2017
78751,Tesouro Prefixado com Juros Semestrais,01/01/2025,02/01/2015,"12,34","12,40","876,52","873,57","873,17",2025,2015,Tesouro Prefixado com Juros Semestrais 2025


## 6. Statistical Analysis - Quartiles

In [26]:
df['Tipo Titulo']  + ' ' + df["Data Base"]

,0
0,Tesouro Prefixado 10/01/2025
1,Tesouro Prefixado 10/01/2025
2,Tesouro Prefixado 10/01/2025
3,Tesouro Prefixado 10/01/2025
4,Tesouro IPCA+ com Juros Semestrais 10/01/2025
...,...
78748,Tesouro IGPM+ com Juros Semestrais 02/01/2015
78749,Tesouro Prefixado com Juros Semestrais 02/01/2015
78750,Tesouro Prefixado com Juros Semestrais 02/01/2015
78751,Tesouro Prefixado com Juros Semestrais 02/01/2015


In [27]:
new_column_names = {
    'Data Vencimento': 'Data vencimento do titulo',
    'Data Base': 'Dia analisado',
}

# Renomeando as colunas
df.rename(columns=new_column_names, inplace=True)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78753 entries, 0 to 78752
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Tipo Titulo                78753 non-null  object
 1   Data vencimento do titulo  78753 non-null  object
 2   Dia analisado              78753 non-null  object
 3   Taxa Compra Manha          78753 non-null  object
 4   Taxa Venda Manha           78753 non-null  object
 5   PU Compra Manha            78753 non-null  object
 6   PU Venda Manha             78753 non-null  object
 7   PU Base Manha              78753 non-null  object
 8   ano_vencimento             78753 non-null  object
 9   ano_analisado              78753 non-null  int64 
 10  Nome_titulo                78753 non-null  object
dtypes: int64(1), object(10)
memory usage: 6.6+ MB


In [29]:
df['Taxa Compra Manha'] = df['Taxa Compra Manha'].str.replace(',','.')
df['Taxa Venda Manha'] = df['Taxa Venda Manha'].str.replace(',','.')

In [30]:
df['Taxa Compra Manha'] = pd.to_numeric(df['Taxa Compra Manha'])

In [31]:
df['Taxa Compra Manha'].min()

-3.53

In [32]:
df['df_limite_inferior'] = df['Taxa Compra Manha'].min()
df['df_limite_inferior'] = (df['Taxa Compra Manha'] - df['df_limite_inferior'])/0.1
df['df_limite_inferior'] = df['df_limite_inferior'].apply(np.floor)
df['df_limite_inferior'] = df['df_limite_inferior'] * 0.1 +  df['Taxa Compra Manha'].min()
df['df_limite_superior'] = df['df_limite_inferior'] + 0.1
df['categoria'] = '[' + df['df_limite_inferior'].round(2).apply(str) + ' - ' + df['df_limite_superior'].round(2).apply(str) + ']'
df['categoria']

,categoria
0,[15.07 - 15.17]
1,[15.27 - 15.37]
2,[14.87 - 14.97]
3,[15.17 - 15.27]
4,[7.27 - 7.37]
...,...
78748,[6.07 - 6.17]
78749,[12.37 - 12.47]
78750,[12.97 - 13.07]
78751,[12.27 - 12.37]


## 7. Exporting Data to Google Sheets

In [33]:
df = df[['Nome_titulo', 'Tipo Titulo',"Taxa Compra Manha","Taxa Venda Manha","PU Compra Manha","PU Venda Manha", "Data vencimento do titulo","Dia analisado", 'categoria']]
df
# "Tipo Titulo","Data vencimento do titulo","Dia analisado","Taxa Compra Manha","Taxa Venda Manha","PU Compra Manha","PU Venda Manha","PU Base Manha","ano_vencimento","ano_analisado","Nome_titulo"

,Nome_titulo,Tipo Titulo,Taxa Compra Manha,Taxa Venda Manha,PU Compra Manha,PU Venda Manha,Data vencimento do titulo,Dia analisado,categoria
0,Tesouro Prefixado 2031,Tesouro Prefixado,15.08,15.20,"434,86","431,94",01/01/2031,10/01/2025,[15.07 - 15.17]
1,Tesouro Prefixado 2027,Tesouro Prefixado,15.31,15.43,"756,34","754,37",01/01/2027,10/01/2025,[15.27 - 15.37]
2,Tesouro Prefixado 2026,Tesouro Prefixado,14.92,15.04,"873,53","872,16",01/01/2026,10/01/2025,[14.87 - 14.97]
3,Tesouro Prefixado 2029,Tesouro Prefixado,15.19,15.31,"572,79","570,12",01/01/2029,10/01/2025,[15.17 - 15.27]
4,Tesouro IPCA+ com Juros Semestrais 2055,Tesouro IPCA+ com Juros Semestrais,7.32,7.44,"3768,88","3711,94",15/05/2055,10/01/2025,[7.27 - 7.37]
...,...,...,...,...,...,...,...,...,...
78748,Tesouro IGPM+ com Juros Semestrais 2021,Tesouro IGPM+ com Juros Semestrais,6.12,6.18,"3073,37","3064,35",01/04/2021,02/01/2015,[6.07 - 6.17]
78749,Tesouro Prefixado com Juros Semestrais 2023,Tesouro Prefixado com Juros Semestrais,12.43,12.49,"887,58","884,96",01/01/2023,02/01/2015,[12.37 - 12.47]
78750,Tesouro Prefixado com Juros Semestrais 2017,Tesouro Prefixado com Juros Semestrais,12.99,13.05,"953,13","952,20",01/01/2017,02/01/2015,[12.97 - 13.07]
78751,Tesouro Prefixado com Juros Semestrais 2025,Tesouro Prefixado com Juros Semestrais,12.34,12.40,"876,52","873,57",01/01/2025,02/01/2015,[12.27 - 12.37]


In [34]:
# df_drive = df.astype(str)

sheet_name = 'Base_tesouro_direto'
gc = gspread.authorize(creds)
worksheet = gc.open(sheet_name).sheet1

# Limpar a planilha antes de atualizar os dados
worksheet.clear()

# Atualizar os dados na planilha existente
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1rOtv9LoEe6RR_OtPkNjzMTsX_BXTwq3t3b5xPstYLGg',
 'updatedRange': "'Página1'!A1:I78754",
 'updatedRows': 78754,
 'updatedColumns': 9,
 'updatedCells': 708786}

## 8. Conclusion

The processed data was exported to Google Sheets and used in a Power BI dashboard. 
The panel clearly visualizes periods of appreciation and depreciation in government bonds, supporting decision-making based on historical behavior.